In [1]:
import pandas as pd
import numpy as np  

df = pd.read_csv("../../data/Data_1/donnees_fusionnees.csv", index_col="datetime", parse_dates=True) # Permet à l'index d'être la colonne datetime et de la convertir en format datetime

battery_capacity = 300 # kWh
battery_power_max = 150 # kW
battery_efficiency = 0.9
eta_charge = 0.95 # rendement de charge
eta_discharge = 0.95 # rendement de décharge
soc = 0 # State Of Charge initial (0%)

df.head()

,power_kW,pv_kW,autoconsommation_kW,injection_kW,soutirage_kW
datetime,,,,,
2023-01-01 00:00:00+00:00,79.357146,0.0,0.0,0.0,79.357146
2023-01-01 01:00:00+00:00,88.694895,0.0,0.0,0.0,88.694895
2023-01-01 02:00:00+00:00,86.957511,0.0,0.0,0.0,86.957511
2023-01-01 03:00:00+00:00,128.153966,0.0,0.0,0.0,128.153966
2023-01-01 04:00:00+00:00,84.119995,0.0,0.0,0.0,84.119995


In [2]:
# Création de listes pour stocker les résultats à chaque heure de simulation
soc_list = []
battery_charge = []
battery_discharge = []
import_after_battery = []
export_after_battery = []

for _, row in df.iterrows(): # itérer sur chaque ligne du DataFrame (chaque heure)
    surplus = row["pv_kW"] - row["power_kW"]

    charge = 0
    discharge = 0
    grid_import = 0 # Importation du réseau après utilisation de la batterie
    grid_export = 0 # Exportation vers le réseau après utilisation de la batterie

    if surplus > 0:  # Excès de production
        # Charger la batterie
        charge = min(surplus, battery_power_max, (battery_capacity - soc) / eta_charge)
        soc += charge * eta_charge  # Prendre en compte l'efficacité
        grid_export = max(surplus - charge, 0)  # Exporter le surplus restant
    else:  # Déficit de production
        # Décharger la batterie
        deficit = -surplus
        discharge = min(deficit / eta_discharge, battery_power_max, soc)
        soc -= discharge
        supplied = discharge * eta_discharge
        grid_import = max(deficit - supplied, 0)  # Importer le reste du déficit
    
    soc_list.append(soc) # Stocker le SOC à chaque heure
    battery_charge.append(charge) # Stocker la charge de la batterie à chaque heure
    battery_discharge.append(discharge) # Stocker la décharge de la batterie à chaque heure

    import_after_battery.append(grid_import) # Stocker l'importation du réseau après utilisation de la batterie
    export_after_battery.append(grid_export) # Stocker l'exportation vers le réseau après utilisation de la batterie


In [3]:
df["soc"] = soc_list
df["import_after_battery"] = import_after_battery
df["export_after_battery"] = export_after_battery


In [4]:
new_import = df["import_after_battery"].sum() # Total de l'importation du réseau après utilisation de la batterie
old_import = df["soutirage_kW"].sum() # Total de l'importation du réseau avant utilisation de la batterie

reduction = (old_import - new_import) / old_import

print(f"Réduction import réseau : {reduction:.2%}")
print("SOC max :", df["soc"].max())
print("SOC final :", df["soc"].iloc[-1])
print("Nombre d'heures où SOC > 0 :", (df["soc"] > 0).sum())

df.head()
df.to_csv("../../data/Data_1/battery_data.csv", index = True)

Réduction import réseau : 0.67%
SOC max : 300.0
SOC final : 0.0
Nombre d'heures où SOC > 0 : 377
